In [63]:
import os
import openai
import requests
import json
from openai import OpenAI
import re
from datetime import datetime, timezone

# SETTINGS
api_key = "Here is your API key"
client = OpenAI(api_key=api_key)


# USER DATA
user_request = """
Ad name with highest clicks across facebook, apple ads and snapchat in 2022?
"""
test_type = 'tables_with_dates'

reports_folder = 'reports'
temperature_setup = 0
max_tokens_setup = 4000
top_p_setup = 1


data_source_file = 'datasource_dates.txt'
guideline_file = 'guideline4.txt'
current_time = datetime.now(timezone.utc)
test_time = current_time.strftime('%Y-%m-%d %H:%M:%S UTC')


# REPORT FILENAME SETUP
matching_files = [file for file in os.listdir(reports_folder) if file.startswith(f'test_{test_type}_')]
if matching_files:
    max_debug_number = max([int(re.search(r'\d+', file).group()) for file in matching_files], default=0)
    new_debug_number = f'{max_debug_number + 1:04d}'
else:
    new_debug_number = '0001'
report_file = f'./reports/test_{test_type}_{new_debug_number}.txt'


# PREPARING SENDING DATA
with open(data_source_file, 'r', encoding='utf-8') as file:
    data_source = file.read()
json_data = json.loads(data_source)
with open(guideline_file, 'r', encoding='utf-8') as file:
    guideline = file.read()
system_prompt = guideline + f'\n' + f'<data_source>\n' + data_source + f'\n</data_source>\n'


# API RESPONSE
response = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[
    {
      "role": "system",
      "content": system_prompt
    },
    {
      "role": "user",
      "content": user_request
    }
  ],
  temperature = temperature_setup,
  max_tokens = max_tokens_setup,
  top_p = top_p_setup
)
response_text = response.choices[0].message.content
# print(f'1st response complete...\n{response_text}\n')


# JSON PARSING PYTHON
text_result = re.split(r'```', response_text)
if text_result:
    try:
        json_text = text_result[1]
        final_result = re.split(r'json', json_text)
        final_json = final_result[1]
    except:
        final_json = "{}"    
else:
    final_json = "{}"
try:
    json_object = json.loads(final_json)
except:
    json_object = {}


# REPORTING
with open(report_file, 'w', encoding='utf-8') as report:
    report.write(f"Test type: {test_type}\nDebug number: {new_debug_number}\n")
    report.write(f"Test date: {test_time}\n")
    report.write(f"Temperature: {temperature_setup}\nMax tokens: {max_tokens_setup}\nTop_p: {top_p_setup} \n")
    report.write(f"============================================================================================================================== \n")
    report.write(f"User Request: \n")
    report.write(f"------------------------------------------------------------------------------------------------------------------------------ \n")
    report.write(f"{user_request}\n")
    report.write(f"============================================================================================================================== \n")
    report.write(f"Response Text: \n")
    report.write(f"cutted: \n")
    # report.write(f"{response_cutted}\n")
    report.write(f"{json_object}\n")
    report.write(f"------------------------------------------------------------------------------------------------------------------------------ \n")
    report.write(f"full: \n")
    report.write(f"{response_text}\n")
    report.write(f"============================================================================================================================== \n")
    #report.write(f"System_prompt (guideline only): \n")
    report.write(f"System_prompt: \n")
    report.write(f"------------------------------------------------------------------------------------------------------------------------------ \n")
    #report.write(f"{guideline}\n")
    report.write(f"{system_prompt}\n")
    
print(f'Report file: {report_file} \n Cutted response: \n{json_object}\n Full response: \n {response_text}')



Report file: ./reports/test_tables_with_dates_0007.txt 
 Cutted response: 
{'data': [{'tab_id': 2, 'pros': 'Contains ad names and clicks for the year 2022.', 'freshness_date': '2023-03-10', 'score': '7', 'table_name': 'ads_6937_facebook_all_data'}]}
 Full response: 
 To find the ad name with the highest clicks across Facebook, Apple Ads, and Snapchat in 2022, we need to look for tables that contain data for the year 2022, have a column for ad names, and a column for clicks. We will also prioritize tables with the most recent `freshness_date` and ensure that the `actual_date_start` and `actual_date_end` cover the year 2022.

Let's analyze the data source to find the most suitable tables for this query.

After analyzing the data source, here are the tables that match the criteria:

For Facebook:
- "table_name": "ads_6937_facebook_all_data"
- "freshness_date": "2023-03-10"
- "actual_date_start": "2021-02-10"
- "actual_date_end": "2023-03-10"
- Contains columns for ad names and clicks.

Fo